In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('JoinedEvent2_CB2016_subset_800m.csv')

In [3]:
weather = pd.read_csv('weather_data_nyc_centralpark_2016(1).csv')

In [4]:
weather.head()

,date,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,1-1-2016,42,34,38.0,0.00,0.0,0
1,2-1-2016,40,32,36.0,0.00,0.0,0
2,3-1-2016,45,35,40.0,0.00,0.0,0
3,4-1-2016,36,14,25.0,0.00,0.0,0
4,5-1-2016,29,11,20.0,0.00,0.0,0


In [32]:
w = weather[['date','average temperature','precipitation']]

In [33]:
w.head()

,date,average temperature,precipitation
0,2016-01-01,38.0,0.00
1,2016-02-01,36.0,0.00
2,2016-03-01,40.0,0.00
3,2016-04-01,25.0,0.00
4,2016-05-01,20.0,0.00


In [16]:
data.head()

,Unnamed: 0,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,...,gender,startdate,stopdate,Event_type,End_Time,weekday,O_date,O_hour,date,average temperature
0,0,173,16:03:10,16:06:03,243,Fulton St & Rockwell Pl,40.688226,-73.979382,241,DeKalb Ave & S Portland Ave,...,2,2016-01-01,2016-01-01,no-event,NaN,NaN,2016-01-01,16,2016-01-01,38.0
1,1,136,16:05:54,16:08:11,420,Clermont Ave & Lafayette Ave,40.687645,-73.969689,270,Adelphi St & Myrtle Ave,...,1,2016-01-01,2016-01-01,no-event,NaN,NaN,2016-01-01,16,2016-01-01,38.0
2,2,653,16:13:47,16:24:40,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,278,Concord St & Bridge St,...,1,2016-01-01,2016-01-01,no-event,NaN,NaN,2016-01-01,16,2016-01-01,38.0
3,3,659,16:13:47,16:24:46,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,278,Concord St & Bridge St,...,2,2016-01-01,2016-01-01,no-event,NaN,NaN,2016-01-01,16,2016-01-01,38.0
4,4,1419,16:20:39,16:44:19,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,532,S 5 Pl & S 4 St,...,1,2016-01-01,2016-01-01,no-event,NaN,NaN,2016-01-01,16,2016-01-01,38.0


In [45]:
d = pd.DataFrame(data.groupby(['O_date','Event_type','O_hour'],as_index=False).size())
d.reset_index(inplace=True)
d.columns = ['O_date','Event_type','O_hour','Count']
d.head()

,O_date,Event_type,O_hour,Count
0,2016-01-01,no-event,16,8
1,2016-01-01,no-event,17,7
2,2016-01-01,no-event,18,4
3,2016-01-01,no-event,19,2
4,2016-01-01,no-event,20,2


In [46]:
w['date'] = pd.to_datetime(w.date)
d['O_date'] = pd.to_datetime(d.O_date) 

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [47]:
d = d.merge(w,left_on='O_date',right_on='date',how='left')

In [48]:
d['O_month'] = pd.to_datetime(d.O_date).dt.month

In [49]:
d = d[d['O_hour']>18]

In [50]:
d.head()

,O_date,Event_type,O_hour,Count,date,average temperature,precipitation,O_month
3,2016-01-01,no-event,19,2,2016-01-01,38.0,0.00,1
4,2016-01-01,no-event,20,2,2016-01-01,38.0,0.00,1
5,2016-01-01,no-event,21,1,2016-01-01,38.0,0.00,1
6,2016-01-01,no-event,22,4,2016-01-01,38.0,0.00,1
7,2016-01-01,no-event,23,3,2016-01-01,38.0,0.00,1


In [51]:
d.rename(columns={'average temperature':'Temperature'},inplace=True)

In [52]:
d['precipitation'] = pd.to_numeric(d['precipitation'],errors='coerce')

In [24]:
import statsmodels.formula.api as smf

In [53]:
lm = smf.ols(formula="Count ~ O_hour + O_month + Temperature + precipitation + C(Event_type, Treatment(reference='no-event'))",data=d).fit()

In [54]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Count   R-squared:                       0.451
Model:                            OLS   Adj. R-squared:                  0.447
Method:                 Least Squares   F-statistic:                     136.9
Date:                Sat, 09 Dec 2017   Prob (F-statistic):          6.53e-209
Time:                        18:17:30   Log-Likelihood:                -6369.2
No. Observations:                1680   AIC:                         1.276e+04
Df Residuals:                    1669   BIC:                         1.282e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
================================================================================================================================
                                                                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       86.9889      4.038     21.543      0.000      79.069      94.909
C(Event_type, Treatment(reference='no-event'))[T.basketball]     1.2352      0.810      1.525      0.127      -0.353       2.824
C(Event_type, Treatment(reference='no-event'))[T.boxing]        -2.5840      1.754     -1.473      0.141      -6.024       0.856
C(Event_type, Treatment(reference='no-event'))[T.concert]        1.0429      0.992      1.052      0.293      -0.902       2.988
C(Event_type, Treatment(reference='no-event'))[T.family]         2.9760      1.469      2.026      0.043       0.096       5.857
C(Event_type, Treatment(reference='no-event'))[T.hockey]         3.4109      1.062      3.213      0.001       1.329       5.493
C(Event_type, Treatment(reference='no-event'))[T.other]         -0.8181      1.856     -0.441      0.659      -4.458       2.822
O_hour                                                          -4.3232      0.186    -23.267      0.000      -4.688      -3.959
O_month                                                          1.9377      0.084     23.148      0.000       1.774       2.102
Temperature                                                      0.1410      0.016      8.858      0.000       0.110       0.172
precipitation                                                   -3.5802      0.896     -3.997      0.000      -5.337      -1.823
==============================================================================
Omnibus:                      405.070   Durbin-Watson:                   1.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1267.078
Skew:                           1.199   Prob(JB):                    7.20e-276
Kurtosis:                       6.514   Cond. No.                         978.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""